# Estimate Multi-ancestry PRS profiles - EUR
- **Project:** Multi-ancestry PRS
- **Version:** Python/3.9
- **Status:** COMPLETE
- **Last Updated:** 16-NOV-2023

## Notebook Overview
- Extract 90 SNPs from individual level data across ancestries in unrelated individual level data
- Merge files
- Estimate profiles


## Extract 90 SNPs from individual level data across ancestries in unrelated individual level data

In [ ]:
###################################### EUR ######################################

In [ ]:
## Input data
cd ${GP2_release5}/imputed_genotypes/EUR/

In [ ]:
## Output data 
mkdir ${WORK_DIR}/EUR/

In [ ]:
## Load packages
module load plink/2.0-alpha

In [ ]:
## Extract 90 risk loci per chr
for chnum in {1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22};
do
    plink2 \
    --pfile chr"$chnum"_EUR_release5 \
    --extract range ${WORK_DIR}/CHR_POS_HG38.txt  \
    --make-bed \
    --out ${WORK_DIR}/EUR/PRS_chr"$chnum"_release5
done

In [ ]:
## Update sex and remove related
module load plink/1.9
cd ${WORK_DIR}
for chnum in {1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22};
do
    plink \
    --bfile EUR/PRS_chr"$chnum"_release5 \
    --update-sex sex.txt \
    --keep related_EUR \
    --make-bed \
    --out EUR/PRS_chr"$chnum"_release5_gender
done

In [ ]:
## Cat files
cd ${WORK_DIR}/EUR/
ls *_release5_gender.bim > temp
head temp

In [ ]:
## Remove *bim so you only keep the core name
sed 's/.bim//g' temp > temp2
head temp2

In [ ]:
## Remove chr1 from the list so we do not merge it twice
grep -v "PRS_chr1_release5_gender" temp2 > files.txt
cat files.txt

## Merge files

In [ ]:
## Merge list using chr 1 as your core input
module load plink/1.9
plink \
--bfile PRS_chr5_release5_gender \
--merge-list files.txt \
--make-bed \
--out PRS_ALL_release5_gender

In [ ]:
## Reformat *bim file (concatenate first and fourth columns and replace second)
awk '{print $1, $1":"$4, $3, $4, $5, $6}' PRS_ALL_release5_gender.bim > temp3
head temp3

In [ ]:
## Replace temp2 by new bim file
mv temp3 PRS_ALL_release5_gender.bim

## Estimate profiles

In [ ]:
## Calculate scores
module load plink/1.9
plink \
--bfile PRS_ALL_release5_gender \
--score ${WORK_DIR}/90LOCI_AFRICAN_CHR_POS.txt \
--out PRS_score_release_AFRICAN

plink \
--bfile PRS_ALL_release5_gender \
--score ${WORK_DIR}/META_FOO_23ANDME_90LOCI_EASTASIAN_CHR_POS.txt \
--out PRS_score_release_EASTASIANS

plink \
--bfile PRS_ALL_release5_gender \
--score ${WORK_DIR}/META_LOESCH_23ANDME_90LOCI_LATINO_CHR_POS.txt \
--out PRS_score_release_LATINO

plink \
--bfile PRS_ALL_release5_gender \
--score ${WORK_DIR}/90LOCI_EUROPEANS_CHR_POS.txt \
--out PRS_score_release_EUROPEANS

In [ ]:
ls ${WORK_DIR}/EUR/